In [8]:
import polars as pl

In [17]:
df = pl.scan_parquet("filer/kjoretoyinfo_fra_2000.parquet").head(10).collect()

In [18]:
df

tekn_reg_f_g,tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_merke,tekn_modell,tekn_farge,tekn_drivstoff,tekn_siste_pkk,tekn_neste_pkk,tekn_aksler_drift,tekn_sitteplasser_totalt
i64,i64,i64,i64,str,str,str,str,str,i64,i64
20211013,20211013,20211013,3900,"""VITO""","""05""","""2""",null,"""20251013""",2,3
20200129,20200129,20200129,5750,"""CADDY""","""06""","""2""","""20231205""","""20260129""",2,2
20231129,20231129,20231129,2600,"""Mustang Mach-E…","""06""","""5""",null,"""20271129""",1,5
20200127,20200127,20200127,5750,"""CADDY""","""01""","""2""",null,"""20240127""",2,2
20180731,20190130,20190130,2600,"""F-150 Platinum…","""06""","""2""","""20220503""","""20240503""",2,2
20200519,20200519,20200519,5750,"""CRAFTER""","""01""","""2""",null,"""20240519""",1,3
20200911,20200911,20220216,5150,"""CITIGO""","""01""","""5""",null,"""20240911""",1,4
20220512,20220512,20220512,5480,"""YARIS""","""04""","""1,5""",null,"""20260512""",1,5
20221104,20221104,20221104,5480,"""RAV4""","""06""","""1,5,5""",null,"""20261104""",2,5


In [ ]:
df = df.with_column(df['tekn_reg_f_g_n'].cast(pl.Int32))